In [ ]:
!wget http://cb.lk/covid_19

--2021-06-07 14:42:04--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 172.67.187.81, 104.21.72.203, 2606:4700:3032::ac43:bb51, ...
Connecting to cb.lk (cb.lk)|172.67.187.81|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2021-06-07 14:42:04--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|172.67.187.81|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2021-06-07 14:42:04--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.66.18, 2620:100:6020:18::a27d:4012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.66.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2021

In [ ]:
!unzip covid_19

Archive:  covid_19
  inflating: CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg  
  inflating: CovidDataset/Val/Covid/85E52EB3-56E9-4D67-82DA-DEA247C82886.jpeg  
  inflating: CovidDataset/Val/Covid/4-x-day1.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-53.jpg  
  inflating: CovidDataset/Val/Covid/4e43e48d52c9e2d4c6c1fb9bc1544f_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-35-1.jpg  
  inflating: CovidDataset/Val/Covid/53EC07C9-5CC6-4BE4-9B6F-D7B0D72AAA7E.jpeg  
  inflating: CovidDataset/Val/Covid/6CB4EFC6-68FA-4CD5-940C-BEFA8DAFE9A7.jpeg  
  inflating: CovidDataset/Val/Covid/58cb9263f16e94305c730685358e4e_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/6b3bdbc31f65230b8cdcc3cef5f8ba8a-40ac-0.jpg  
  inflating: CovidDataset/Val/Covid/auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg  
  inflating: CovidDataset/Val/Covid/6b44464d-73a7-4cf3-bbb6-ffe7

In [ ]:
train_path = 'CovidDataset/Train'
val_path = 'CovidDataset/Val'
test_predict = 'test'

In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.layers import *
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.models import *
import pickle

In [ ]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation = 'relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss=keras.losses.binary_crossentropy,optimizer='adam',metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 128)     73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 128)       0

In [ ]:
train_data=image.ImageDataGenerator(
    rescale = 1/255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True
)
test_data=image.ImageDataGenerator( rescale = 1/255)

In [ ]:
train_genrator = train_data.flow_from_directory(
    'CovidDataset/Train',target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)
print(type(train_data))

Found 224 images belonging to 2 classes.
<class 'keras.preprocessing.image.ImageDataGenerator'>


In [ ]:
train_genrator.class_indices

{'Covid': 0, 'Normal': 1}

In [ ]:
val_gen = test_data.flow_from_directory(
    'CovidDataset/Val',target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 60 images belonging to 2 classes.


In [ ]:
hist = model.fit(
    train_genrator,
    steps_per_epoch=7,
    epochs=10,
    validation_data=val_gen,
    validation_steps=2
)

Epoch 1/10
7/7 [==============================] - 61s 2s/step - loss: 4.4132 - accuracy: 0.5042 - val_loss: 0.6793 - val_accuracy: 0.9167
Epoch 2/10
7/7 [==============================] - 11s 2s/step - loss: 0.7120 - accuracy: 0.5439 - val_loss: 0.6629 - val_accuracy: 0.8833
Epoch 3/10
7/7 [==============================] - 11s 2s/step - loss: 0.5457 - accuracy: 0.7444 - val_loss: 0.3882 - val_accuracy: 0.9333
Epoch 4/10
7/7 [==============================] - 11s 2s/step - loss: 0.4116 - accuracy: 0.8011 - val_loss: 0.2381 - val_accuracy: 0.9333
Epoch 5/10
7/7 [==============================] - 11s 2s/step - loss: 0.3602 - accuracy: 0.8523 - val_loss: 0.2667 - val_accuracy: 0.9500
Epoch 6/10
7/7 [==============================] - 11s 2s/step - loss: 0.3177 - accuracy: 0.8929 - val_loss: 0.1591 - val_accuracy: 0.9500
Epoch 7/10
7/7 [==============================] - 11s 2s/step - loss: 0.3234 - accuracy: 0.8577 - val_loss: 0.1695 - val_accuracy: 0.9833
Epoch 8/10
7/7 [==================

In [ ]:
model.save('model.h5')

In [ ]:
model.evaluate(train_genrator)

7/7 [==============================] - 8s 1s/step - loss: 0.0764 - accuracy: 0.9732


[0.07642291486263275, 0.9732142686843872]

In [ ]:
model.evaluate(val_gen)

2/2 [==============================] - 2s 967ms/step - loss: 0.0661 - accuracy: 0.9667


[0.06606336683034897, 0.9666666388511658]

In [ ]:
img=image.load_img('test/11.jpg',target_size=(224,224))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
model.predict(img_data)

array([[0.]], dtype=float32)